In [74]:
from newencoder import Encoder
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
W2V_PATH = "/home/jingjing/Desktop/InferSent-master/dataset/GloVe/glove.840B.300d.txt"

In [3]:
f = Encoder()
f.set_w2v_path(W2V_PATH)

In [4]:
def make_target(context_size, dim):
    targets = np.zeros((dim, dim))
    ctxt_sent_pos = list(range(-context_size, context_size+1))
    ctxt_sent_pos.remove(0)
    for ctxt in ctxt_sent_pos:
        targets += np.eye(3, k=ctxt)
    targets_sum = np.sum(targets,axis=1, keepdims=True)
    targets = targets / targets_sum
    targets = torch.from_numpy(targets)
    return targets

In [5]:
def make_senmat(enc, sentences, bsize, tokenize, verbose):
    enc.build_vocab(sentences, True)
    embeddings = enc(sentences, bsize, tokenize, verbose)
    scores = np.matmul(embeddings,np.transpose(embeddings))
    scores_sum = np.sum(scores, axis=1, keepdims=True)
    scores = scores/scores_sum
    scores = torch.from_numpy(scores)
    scores.requires_grad = True
    return scores

In [6]:
def xentropy_cost(pred, target):
    logged = torch.log(pred)
    a = target.float()*logged
    cost = -torch.sum(a)
    return cost

In [66]:
weight = torch.FloatTensor(3).fill_(1)
def loss_fn(pred, target):
    m = nn.Sigmoid()
    s_pred = m(pred)
    losses = F.binary_cross_entropy_with_logits(s_pred, target, weight=weight)
    return losses

In [67]:
data = [['Memories of childhood are unforgettable.', 'I was four years old when my grandfather died.',
             'I clearly remember how everybody in the house was weeping.'], ['Today is sunny', 'We should go out for a picnic.', 'Love the weather.']]

In [68]:
test = ['Doctors lead a hard life.', 'Their life is very busy.', 'They get up early in the morning and go to the hospital.']

In [69]:
with torch.no_grad():
    f.build_vocab(test, True)
    scores = f(test, 400, False, True)
    print(scores)
    targets = make_target(1, 3)
    print(targets)
    loss = loss_fn(scores, targets.float())
    print(loss)

Found 23(/23) words with w2v vectors
Vocab size : 23
Nb words kept : 25/28 (89.3%)
Speed : 110.1 sentences/s (cpu mode, bsize=400)
tensor([[ 0.3526,  0.3160,  0.3315],
        [ 0.3181,  0.3541,  0.3279],
        [ 0.3233,  0.3176,  0.3591]])
tensor([[ 0.0000,  1.0000,  0.0000],
        [ 0.5000,  0.0000,  0.5000],
        [ 0.0000,  1.0000,  0.0000]], dtype=torch.float64)
tensor(0.8333)


In [70]:
optimizer = optim.Adam(f.parameters(), lr=0.1, weight_decay=0.1)


In [79]:
for epoch in range(1):
    for instance in data:
        f.zero_grad()
        scores = f(instance, 400, False, False)
        scores = Variable(scores, requires_grad=True)
        targets = make_target(1,3)
        loss = loss_fn(scores, targets.float())
        print(epoch, loss)
        loss.backward()
        optimizer.step()
        for i in f.parameters():
            print(i.grad)

0 tensor(0.8313)
None
None
None
None
None
None
None
None
0 tensor(0.8321)
None
None
None
None
None
None
None
None


In [72]:
with torch.no_grad():
    scores = f(test, 400, False, True)
    print(scores)
    targets = make_target(1, 3)
    print(targets)
    loss = loss_fn(scores, targets.float())
    print(loss)

Nb words kept : 25/28 (89.3%)
Speed : 77.2 sentences/s (cpu mode, bsize=400)
tensor([[ 0.3526,  0.3160,  0.3315],
        [ 0.3181,  0.3541,  0.3279],
        [ 0.3233,  0.3176,  0.3591]])
tensor([[ 0.0000,  1.0000,  0.0000],
        [ 0.5000,  0.0000,  0.5000],
        [ 0.0000,  1.0000,  0.0000]], dtype=torch.float64)
tensor(0.8333)
